# Лабораторная работа № 4. Модель однофазной многоканальной замкнутой системы обслуживания
Выполнил Пакало Александр Сергеевич, студент РТ5-81Б

Вариант № 5

## Задание № 1
Вычислительный центр фирмы состоит из $m$
 главных серверов коллективного пользования. Число работающих в центре
 программистов в любой момент времени равно
$k$. Каждый программист готовит свою программу и через терминал передает ее
на сервер для выполнения, куда она сразу попадает. Время подготовки программ
имеет экспоненциальное распределение со средним значением
$t_1$ мин. Время выполнения программы на любом из серверов имеет экспоненциальное
 распределение со средним значением
 $t_2$ мин. Каждый программист ожидает ответа от сервера, прежде, чем начнет писать следующую программу.

Найти (теоретически и экспериментально):
1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.
2. Среднее время до получения пользователем результатов реализации.
3. Среднее количество программ, ожидающих выполнения на сервере.

Требуется выполнить расчет при заданном по варианту значении
$m$, а также при $m = 1$.

In [20]:
Variant <- 5
set.seed(Variant)
k <- sample(c(10:25), 1)
m <- sample(c(3:6), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:6), 1)
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
11,5,14,4


Найдем интенсивность выполнения программ:

In [21]:
lambda_1 <- 1 / t1
lambda_1

[1] 0.07142857

Интенсивность поступления программ от $k$ программистов:

In [22]:
lambda_k <- k * lambda_1
lambda_k

[1] 0.7857143

Интенсивность выполнения программ:

In [23]:
miu <- 1 / t2
miu

[1] 0.25

Интенсивность выполнения программ меньше интенсивности поступления,
следовательно очередь в нашей системе будет бесконечно расти. Согласно
указаниям преподавателя, изменим начальные параметры так, чтобы неравенство
обратилось:

In [202]:
t1 <- 45
lambda_k <- k / t1
lambda_k

[1] 0.2444444

Итого имеем следующие начальные величины:

In [203]:
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<dbl>,<int>
11,5,45,4


In [ ]:
# Зададим таблицу результатов.

In [204]:
results <- data.frame(probability = c("-", "-"), T = c("-", "-"), Lqueue = c("-", "-"))
row.names(results) <- c('theoretical', 'practical')
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,-,-,-
practical,-,-,-


### Теоретически

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
Мы имеем дело с многоканальной **замкнутой** СМО с неограниченной очередью
$M / M / m / \infty$. Следовательно:

$$
P_0 = \left(
  \sum_{i = 0}^m \frac{k! \rho^i}{\left(k - i \right)!i!}
  +
  \sum_{i = 1}^{k - m} \frac{k! \rho^{m + i}}{m! \cdot (k - m - i)!m^i}
\right) ^ {-1}
$$
где $\rho = \frac{\lambda}{\mu}$

In [205]:
ro <- lambda_1 / miu
ro

[1] 0.2857143

In [206]:
i1 <- 0:m
parts_of_sum1 <- c(factorial(k) * ro^i1 / (factorial((k - i1)) * factorial(i1)))

i2 <- 1:(k - m)
parts_of_sum2 <- c(factorial(k) * ro^(m + i2) / (factorial(m) * factorial((k - m - i2)) * m^i2))

P0 <- (sum(parts_of_sum1) + sum(parts_of_sum2))^(-1)
P0

[1] 0.06261347

In [207]:
P0 * k * ro

[1] 0.1967852

Тогда вероятность того, что $i$ серверов занято:
$$
P_i = P_0 \cdot \frac{k! \rho^i}{\left(k - i \right)!i!}
$$

Для этого воспользуемся уже созданным массивом частей суммы:

In [208]:
get_Pi <- function(part) part * P0

In [209]:
Pi <- lapply(parts_of_sum1[1:m], get_Pi)
Pi

[[1]]
[1] 0.06261347

[[2]]
[1] 0.1967852

[[3]]
[1] 0.2811217

[[4]]
[1] 0.2409615

[[5]]
[1] 0.1376923

Тогда $\sum_{i = 0}^{m} P_i$ -  вероятность того, что хотя бы один сервер
будет доступен. Следовательно, обратная вероятность и будет нашим ответом.

In [375]:
probability <- 1 - sum(unlist(Pi))
results$probability[1] <- probability
probability

[1] 0.08082587

#### 2. Среднее время до получения пользователем результатов реализации.
По формуле Литтла, подходящей для СМО любого вида имеем:
$$
T_{\text{сист}} = \frac{L_{\text{сист}}}{\Lambda} \\
\Lambda = (k - L_{\text{сист}}) \cdot \lambda \\
L_{\text{сист}} = \sum_{i = 1}^k \left(P_i \cdot i \right)
$$

In [211]:
Pi_queue <- mapply(function(Pi, i) Pi * i, Pi[1:k], 1:k)
Lsys <- sum(unlist(Pi))
Lsys

[1] 0.9191741

In [212]:
Lambda <- (k - Lsys) * lambda_1
Lambda

[1] 0.720059

In [213]:
T <- Lsys / Lambda
results$T[1] <- T
T

[1] 1.276526

#### 3. Среднее количество программ, ожидающих выполнения на сервере.
Она же средняя длина очереди:
$$
L_{\text{оч}} = \frac{\rho^{m+1}}{m! \cdot m} \cdot P_0 \cdot
\frac{1}{\left( 1 - \frac{\rho}{m} \right)^2}
$$

In [214]:
Lqueue <- ro^(m + 1) / (factorial(m) * m) * P0 * 1 / (1 - ro / m)^2
results$Lqueue[1] <- Lqueue
Lqueue

[1] 6.385817e-08

In [215]:
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.0808258656604589,1.27652615492436,6.38581729495202e-08
practical,-,-,-


Как видно, очередь система работает достаточно быстро, чтобы не накапливать очередь.
Это подтверждается и при сравнении интенсивностей поступления / обработки программ.

### Численно

In [278]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

SIMULATION_TIME <- 10000

env <- simmer("SupaDupaSim")

programs <- trajectory() %>%
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t1)) %>%
    release("server", 1)

env %>%
    add_resource("server", capacity = m, queue_size = k - m) %>%
    add_generator(
        "programs",
        programs,
        distribution = function() rexp(1, lambda_k)
    ) %>%
    run(until = SIMULATION_TIME)

simmer environment: SupaDupaSim | now: 10000 | next: 10000.8185453627
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 5(5) | queue status: 5(6) }
{ Source: programs | monitored: 1 | n_generated: 2372 }

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.

In [292]:
resources <- get_mon_resources(env)
resources

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,7.074665,1,0,5,6,1,11,1
server,10.818545,2,0,5,6,2,11,1
server,12.748086,3,0,5,6,3,11,1
server,13.705893,4,0,5,6,4,11,1
server,17.441328,5,0,5,6,5,11,1
server,17.879268,5,1,5,6,6,11,1
server,20.928459,5,2,5,6,7,11,1
server,22.757949,5,3,5,6,8,11,1
server,24.259523,5,4,5,6,9,11,1


In [368]:
probability <- resources %>% with(sum(server == m) / length(server)) / 2
results$probability[2] <- probability

#### 2. Среднее время до получения пользователем результатов реализации.

In [369]:
arrivals <- get_mon_arrivals(env)
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,11.18959,28.86244,17.67285,TRUE,1
programs12,52.71184,52.71184,0.00000,FALSE,1
programs13,53.70097,53.70097,0.00000,FALSE,1
programs14,61.51837,61.51837,0.00000,FALSE,1
programs15,63.05941,63.05941,0.00000,FALSE,1
programs16,68.87075,68.87075,0.00000,FALSE,1
programs1,21.15970,71.38553,42.52309,TRUE,1
programs18,76.97138,76.97138,0.00000,FALSE,1
programs19,78.18275,78.18275,0.00000,FALSE,1


In [370]:
T <-arrivals %>% subset(finished) %>% with(mean(end_time - start_time))
results$T[2] <- T

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

In [371]:
Lqueue <- resources %>% with(mean(queue))
results$Lqueue[2] <- Lqueue

In [372]:
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.0808258656604589,1.27652615492436,0.114285714285714
practical,0,484.169901066834,9.25173210161663


### Для $m = 1$
Зададим таблицу результатов для $m = 1$.

In [366]:
mm1.results <- data.frame(probability = c("-", "-"), T = c("-", "-"), Lqueue = c("-", "-"))
row.names(mm1.results) <- c('theoretical', 'practical')
mm1.results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,-,-,-
practical,-,-,-


### Теоретически для $m = 1$

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.

$$
1 - P_0 = \frac{\lambda}{\mu}
$$

In [354]:
mm1.probability <- lambda_1 / miu
mm1.results$probability[1] <- mm1.probability
mm1.probability

[1] 0.2857143

#### 2. Среднее время до получения пользователем результатов реализации.
Вычислим среднее время до получения пользователем результатов реализации:

$$
T_{\text{сист}} = \frac{1}{\mu(1 - \rho)}
$$

In [355]:
mm1.T <- 1 / (miu * (1 - ro))
mm1.results$T[1] <- mm1.T
mm1.T

[1] 5.6

#### 3. Среднее количество программ, ожидающих выполнения на сервере.
Она же средняя длина очереди:
$$
L_{\text{оч}} = \frac{\rho^{2}}{1 - \rho}
$$

In [356]:
mm1.Lqueue <- ro^2 / (1 - ro)
mm1.results$Lqueue[1] <- mm1.Lqueue
mm1.Lqueue

[1] 0.1142857

In [357]:
mm1.results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.285714285714286,5.6,0.114285714285714
practical,0.297389012783001,5.56810237849018,0.110123789102398


Как видно, очередь система работает достаточно быстро, чтобы не накапливать очередь.
Это подтверждается и при сравнении интенсивностей поступления / обработки программ.

### Численно для $m = 1$

In [358]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

SIMULATION_TIME <- 10000

env <- simmer("SupaDupaSim")

programs <- trajectory() %>%
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t1)) %>%
    release("server", 1)

env %>%
    add_resource("server", capacity = 1, queue_size = k - 1) %>%
    add_generator(
        "programs",
        programs,
        distribution = function() rexp(1, lambda_k)
    ) %>%
    run(until = SIMULATION_TIME)

simmer environment: SupaDupaSim | now: 10000 | next: 10000.1278464687
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 10(10) }
{ Source: programs | monitored: 1 | n_generated: 2477 }

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.

In [359]:
mm1.resources <- get_mon_resources(env)

In [360]:
mm1.probability <- mm1.resources %>% with(sum(server == 1) / length(server))
mm1.results$probability[2] <- mm1.probability

#### 2. Среднее время до получения пользователем результатов реализации.

In [361]:
mm1.arrivals <- get_mon_arrivals(env)

In [362]:
mm1.T <- mm1.arrivals %>% subset(finished) %>% with(mean(end_time - start_time))
mm1.results$T[2] <- T

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

In [363]:
mm1.Lqueue <- mm1.resources %>% with(mean(queue))
mm1.results$Lqueue[2] <- mm1.Lqueue

In [365]:
mm1.results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.285714285714286,5.6,0.114285714285714
practical,0.297389012783001,5.56810237849018,0.110123789102398
